# CS 6080 - Time Series Taxonomy

In [ ]:
import gc
import os
import time

import pandas as pd
import matplotlib as plt
import numpy as np

from pyts.classification import KNeighborsClassifier, BOSSVS

from sklearn.model_selection import StratifiedShuffleSplit

from utils import load_dataset, run_classifiers_on_dataset

## Load Datasets into RAM and Perform learning on the Species and Breed labels

In [ ]:
# dataset_a1_sample_size = 64*64
# dataset_a1 = load_dataset("data/a1", dataset_a1_sample_size)

In [ ]:
# dataset_a1_metrics = run_classifiers_on_dataset(dataset_a1)
# del dataset_a1
# gc.collect()

In [ ]:
dataset_a2_sample_size = 64*3
dataset_a2 = load_dataset("data/a2", dataset_a2_sample_size)

In [ ]:
dataset_a2_metrics = run_classifiers_on_dataset(dataset_a2)
del dataset_a2
gc.collect()

In [ ]:
# dataset_a3_sample_size = 64*3
# dataset_a3 = load_dataset("data/a3", dataset_a3_sample_size)

In [ ]:
# dataset_a3_metrics = run_classifiers_on_dataset(dataset_a3)
# del dataset_a3
# gc.collect()

## Report metrics and create plots (Accuracy, Precision, Recall, ROC)